# Infrastructure and Configuration

## Initialize data

In [ ]:
/* Systemorph set of dimensions + mockdata are dispatched to the unconfigured in-memory DataSource */
#!eval-notebook "../Initialization/InitSystemorphToMemory"
/* DataSource is configured and connected to real database */
//#!eval-notebook "../Database/Configure.ipynb"

In [ ]:
Workspace.InitializeFrom(DataSource);

## Import Dependencies

In [ ]:
//#!import "ReportScopes.ipynb"
//#!import "ReportConfigurationAndUtils.ipynb"

# Report Scope Proposal

In [ ]:
public enum ReportNames { PvReport, FcfReport}
[InitializeScope(nameof(InitAsync))]
public interface IIfrs17Report : IMutableScope<ReportNames,ReportStorage> {
    // Some infrastructure
    protected IWorkspace workspace => GetStorage().Workspace;
    protected Systemorph.Vertex.Pivot.Reporting.IReportFactory report => GetStorage().Report;

    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<IIfrs17Report>(s => s.WithApplicability<FcfReport>(x => x.Identity == ReportNames.FcfReport)
        .WithApplicability<PvReport>(x => x.Identity == ReportNames.PvReport)
                );

    
    // Basic mutable properties
    (int Year, int Month) ReportingPeriod { get; set; }
    int Year => ReportingPeriod.Year;
    int Month => ReportingPeriod.Month;
    string ReportingNode { get; set; }
    string Scenario { get; set; }
    CurrencyType CurrencyType { get; set; }
    
    ((int Year, int Month) ReportingPeriod, string ReportingNode, string Scenario, CurrencyType) ShowSettings => (ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
    // Slice and Dice
    IEnumerable<string> RowSlices { get; set; }
    protected string[] defaultRowSlices => new string[] { };
    protected string[] rowSlices => RowSlices is null ? defaultRowSlices : defaultRowSlices.Concat(RowSlices).ToArray();

    IEnumerable<string> ColumnSlices { get; set; }
    protected string[] defaultColumnSlices => new string[] {  };
    protected string[] columnSlices => ColumnSlices is null ? defaultColumnSlices : defaultColumnSlices.Concat(ColumnSlices).ToArray();
    protected HashSet<(ReportIdentity, CurrencyType)> GetIdentities() =>  GetStorage().GetIdentities(ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
    // Filter
    IEnumerable<(string filterName, string filterValue)> DataFilter { get; set; }
    protected (string filterName, object filterValue)[] dataFilter => (DataFilter is null ? Enumerable.Empty<(string, object)>() : DataFilter.Select(x => (x.filterName, (object)x.filterValue))).ToArray();
    
    // Scope Initialization
    async Task InitAsync() {
        var mostRecentPartition = (await workspace.Query<PartitionByReportingNodeAndPeriod>().Where(x => x.Scenario == null).OrderBy(x => x.Year).ThenBy(x => x.Month).ToArrayAsync()).Last();  
        
        ReportingPeriod = (mostRecentPartition.Year, mostRecentPartition.Month);
        ReportingNode = (await workspace.Query<ReportingNode>().Where(x => x.Parent == null).ToArrayAsync()).First().SystemName; // TODO: change once user permissions are available
        Scenario = null;
        CurrencyType = CurrencyType.Contractual;
        
        ColumnSlices = "EstimateType".RepeatOnce();


        await GetStorage().InitializeReportIndependentCacheAsync();
    }
    int GetNumber() => 10;
    public IDataCube<ReportVariable> GetDataCube() => default;
    
    //Needs to be copy pasted
    private async Task<GridOptions> GetReportTaskAsync() {
        await GetStorage().InitializeAsync(ReportingPeriod, ReportingNode, Scenario, CurrencyType);
    
        
        return report.ForDataCube(GetDataCube())
            .WithQuerySource(workspace)
            .SliceRowsBy(rowSlices)
            .SliceColumnsBy(columnSlices)
            .ReportGridOptions()
            .ToReport();
    }

    Task<GridOptions> ToReportAsync => GetReportTaskAsync();



 }
public interface PvReport : IIfrs17Report {
    string[] IIfrs17Report.defaultRowSlices => new string[] {  "Novelty", "VariableType" };

    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType), "LiabilityType", "EconomicBasis" };

    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() => GetScopes<LockedBestEstimate>(GetIdentities()).Aggregate().LockedBestEstimate.Filter(dataFilter) ;//+ GetScopes<CurrentBestEstimate>(identities).Aggregate().CurrentBestEstimateFcf.Filter(dataFilter);
   
 }

 public interface FcfReport : IIfrs17Report {

    string[] IIfrs17Report.defaultRowSlices => new string[] { "Novelty","VariableType" };

    string[] IIfrs17Report.defaultColumnSlices => new string[] { CurrencyGrouper(CurrencyType) };

    IDataCube<ReportVariable> IIfrs17Report.GetDataCube() =>  GetScopes<Fcf>(GetIdentities()).Aggregate().Fcf.Filter(dataFilter);


 }
    

In [ ]:
public class Ifrs17 
{
    private IWorkspace Workspace {get; init;}
    private Systemorph.Vertex.Scopes.Proxy.IScopeFactory Scopes {get; init;}
    private Systemorph.Vertex.Pivot.Reporting.IReportFactory Report {get; init;}
    private ReportStorage Storage {get; set;}
    public PvReport PresentValues {get{
        Storage = new ReportStorage(Workspace,Report);
        return Scopes.ForSingleton().WithStorage(Storage).ToScope<PvReport>();
    }}
    public Ifrs17 (IWorkspace ws, Systemorph.Vertex.Scopes.Proxy.IScopeFactory scopes, Systemorph.Vertex.Pivot.Reporting.IReportFactory report)
    {
        Workspace = ws; 
        Scopes = scopes; 
        Report = report; 
    }
}

# Playground

In [ ]:
 var reportStorage = new ReportStorage(Workspace, Report);
 var ids = new ReportNames[] {ReportNames.PvReport, ReportNames.FcfReport};
 var reports = Scopes.ForIdentities(ids).WithStorage(reportStorage).ToScopes<IIfrs17Report>();
var fcfReport = reports.Last();



In [ ]:
  fcfReport.ReportingNode = "CH";
 fcfReport.ReportingPeriod = (2020, 12);

 (await fcfReport.ToReportAsync) with {Height = 250}


In [ ]:
fcfReport.Last().GetNumber()

In [ ]:
 var reportStorage = new ReportStorage(Workspace, Report);
 var pvReport = Scopes.ForSingleton().WithStorage(reportStorage).ToScope<PvReport>();
 pvReport.ReportingNode = "CH";
 pvReport.ReportingPeriod = (2020, 12);
 (await pvReport.ToReportAsync) with {Height = 250}



In [ ]:
pvReport.GetNumber()

In [ ]:
typeof(PvReport)

In [ ]:
fcfReport is PvReport

In [ ]:
public interface Hello {
    public int Number();
}
public class Class1 : Hello {
    public int Number() => 5;
}
public class Class2 : Hello
{
    public int Number() => 7;
}

In [ ]:
var c1= new Class1();
var c2 = new Class2();
c1 is Class2